# Core concepts in machine learning
Now that we have the preliminaries out of the way, we're ready to dive in and do some machine learning using scikit-learn. In this notebook, I'll provide a basic overview of some core concepts in machine learning, and then, as a case study, we'll take a look at one particular application of machine learning that you're probably already familiar with, but may not have previously thought of as machine learning: linear regression.

Our first step is to import all of the core tools we're going to use repeatedly throughout this tutorial. This is conventional in Python; the idea is that consolidating all of the imports at the top helps us organize and keep track of all the external libraries and tools we're using. For pedagogical , I'll defer importing specific scikit-learn functionality until the point at which it's actually introduced. That way it'll be clear exactly where everything is coming from.

In [ ]:
# the workhorse numerical computing package in Python
import numpy as np

# we've already encountered this specimen
import pandas as pd

# matplotlib is Python's main plotting library.
# the plt module provides high-level access to plots.
import matplotlib.pyplot as plt

# use the 'bmh' style for all plots
plt.style.use('default')

# this is our data retrieval helper from the first notebook
from support import get_features

Additionally, let's also import the personality data we described in the first notebook.

In [ ]:
data = pd.read_csv('data/Johnson-2014-IPIP-300.tsv.gz', sep='\t', )

## What *is* machine learning?
We're about a quarter of the way into the second notebook of our machine learning tutorial, and we still haven't even defined machine learning! This is probably a good time to do that. Here's a working definition: **machine learning is the field of science/engineering that seeks to build systems capable of learning from experience.**

This is a very broad definition, and in practice, the set of activities that get labeled "machine learning" is pretty heterogeneous. However, two elements are common to nearly all machine learning applications: (a) the emphasis is on developing algorithms that can learn (semi-)autonomously from data, rather than static rule-based systems that must be explicitly designed or updated by humans; and (b) algorithm evaluation focuses heavily on the ability to meet objective quantitative targets.

## The scikit-learn package
Now that we know what machine learning is, let's turn to the scikit-learn package. Scikit-learn is the most widely-used machine learning package in Python (and probably the most widely-used ML package, period). Its popularity stems from its simple, elegant API, [stellar documentation](https://scikit-learn.org/stable/documentation.html), and comprehensive support for many of the most widely used machine learning algorithms (the main exception being deep learning). Scikit-learn provides well-organized, high-quality tools for virtually all aspects of the typical machine learning workflow, including data loading and preprocessing, feature extraction and feature selection, dimensionality reduction, model selection and evaluation, and so on.

### Estimators in scikit-learn
One of scikit-learn's most attractive features is its unified, simple interface for configuring estimators and fitting models. A good deal of the package consists of a very large set of `Estimator` classes you can use to do various forms of machine learning. In scikit-learn, an `Estimator` does exactly what the word *estimator* normally means in statistics: it implements ["a rule for calculating an estimate of a given quantity based on observed data"](https://en.wikipedia.org/wiki/Estimator).

While scikit-learn includes hundreds of different estimators that generate predictions in very different ways, they all share a common interface. In particular, they all implement `.fit()` and `.predict()` methods. When we call an estimator's `.fit()` method, we're telling it to take some training data—including both `X` and `y` values—and learn how to predict `y` from `X`. When we call `predict()`, we're asking the estimator to take some `X` rows (which could be either ones we've seen before, or entirely new ones) and predict corresponding `y` scores.

To see this in action, let's take a look at the implementation of "ordinary" least-squares regression in scikit-learn. Suppose we have two variables—call them `x` and `y`—and we're interested in predicting `y` from `x`:

In [ ]:
import numpy as np

# set random seed for reproducibility
np.random.seed(100)

# generate moderately correlated variables.
X = np.random.normal(size=(40, 1))
y = np.random.normal(X, 2)

plt.scatter(X, y);

Our goal is to identify the line of best fit through this data—i.e., the intercept and slope parameters that minimize the sum of squared errors from the observed data to the regression line. We start by initializing `LinearRegression` estimator:

In [ ]:
# OLS (and many other variants of regression) is housed in the linear_model module
from sklearn.linear_model import LinearRegression

# initialize the model
model = LinearRegression()

The `LinearRegression` estimator, unlike many others, has very few configurable parameters. Above, we initialize it with all of the default values.

Now we're ready to fit some data! We can do that by calling the `.fit()` method. This will be true for every `Estimator` in scikit-learn.

In [ ]:
# Fit the model
model.fit(X, y)

Now we have a fitted model. One thing we can do at this point is examine the estimated model parameters:

In [ ]:
# The sklearn convention is to denote fitted parameters with a trailing underscore
print(model.intercept_)
print(model.coef_)

We can use these parameter estimates to manually construct and apply a prediction equation (i.e., $\hat{y} = -0.557 + 1.43x$) if we want to. But if we just want to generate predicted scores given a set of `X` values (either the same ones we used in training, or new ones), we can make use of the `.predict()` method that all `Estimator` classes implement. For example, here are the predicted scores for our original `X` data:

In [ ]:
y_pred = model.predict(X)

y_pred

Let's make use of `predict()` to generate a regression line we can drop into our scatter plot:

In [ ]:
x_range = np.linspace(X.min(), X.max(), 100)[:, None]
reg_line = model.predict(x_range)

plt.scatter(X, y)
plt.plot(x_range, reg_line);

That's it! In just a few lines of code, we've initialized a linear regression model, fitted it using some data, and generated new predicted scores. This basic pattern is common to all estimators in scikit-learn.

Just to underscore how little we had to do, here's the whole example again, in three lines:

In [ ]:
# Initialize the linear regression estimator
est = LinearRegression()

# Fit the model
est.fit(X, y)

# Generate predictions
y_pred = est.predict(X)

## Supervised learning
Machine learning techniques come in two general flavors: *supervised learning* and *unsupervised learning*. Learning is supervised whenever we know the true values that our model is trying to predict, and hence, are in a position to "supervise" the learning process by quantifying prediction accuracy and making iterative adjustments.

Some examples of supervised learning problems:

* Determining whether or not incoming email is spam
* Predicting a person's age from personality scores
* Diagnosing schizophrenia based on genetic markers

Within the class of supervised learning problems, we can draw a further distinction between *classification* problems and *regression* problems. In both cases, the goal is to develop a predictive model that recovers the true labels as accurately as possible. The difference between the two lies in the nature of the labels: in classification, the labels are discrete; in regression, they're continuous.

### Regression
Let's take the regression first case. As we saw above, a good example of regression, in the machine learning context, is... well, conventional linear regression. "Ordinary" least-squares regression is an example of supervised learning, because our model takes as its input both a vector of *features* (conventionally labeled `X`) and a vector of *labels* (`y`). We often use different terminology in the social and biomedical sciences—calling `X` our set of variables or predictors, and `y` our outcome or dependent variable—but the idea is the same.

Let's return to our running age-prediction example. Predicting age from personality is a regression problem, because age varies continuously. What we're trying to find is some optimal function that, given a matrix of personality scores, can produce a set of continuous values that best approximates (for whatever definition of "best" we like) the true ages of our participants.

There's essentially no difference between our earlier linear regression example and the present one, save for the input data. We have various feature sets available in our data, but let's try to predict age from the Big Five "domains" (i.e., the broadest level of the putative personality hierarchy).

In [ ]:
# get data using our helper
X, y = get_features(data, 'domains', 'AGE')

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

#### Performance metrics
Once we've fit our model, it's natural to want to know how well it performs. At this point, if you're used to a point-and-click package like SPSS, or a stats-oriented language like R, you're probably expecting to see a big dump of information that includes things like regression coefficient estimates and standard errors, p-values, $R^2$ values, and so on. Well... you're not going to get those here. You *can* get those using other Python packages; for example StatsModels will give you [something very similar](https://www.statsmodels.org/dev/regression.html) to R's `lm` summary):

In [ ]:
import statsmodels.formula.api as smf

model = smf.ols('AGE ~ Neuroticism + Extraversion + Openness + Agreeableness + Conscientiousness', data=data)
model.fit().summary()

As I said, you *can* do this kind of thing. But... that's not really what machine learning (or scikit-learn) is about. Instead, the focus is on prediction; typically, we have some objective quantitative metric we care about, and to the degree that a model can produce better values on that metric, we incline to evaluate it more favorably. This doesn't mean that we have to single-mindedly base our evaluation of model on just one quantity; in practice, many other considerations may come into play (e.g., computational efficiency, interpretability, etc.). The point is mainly that machine learning practitioners—at least in applied settings—tend to care much more than traditional scientists do about what models can actually *do*, and much less about what's going on inside them.

##### The coefficient of determination
For the rest of this tutorial, we're going to focus our attention on one particular metric of predictive performance: the coefficient of determination, or $R^2$. $R^2$ quantifies the proportion of variance in the outcome variable (e.g., age) explained by the fitted model:

$$R^2 = 1 - \frac{SS_{residual}}{SS_{total}}$$

$R^2$ is the most widely used measure of performance in the individual differences literature, and we'll stick with tradition here. But this shouldn't be taken as an indication that there's anything particularly special about $R^2$. In fact, in many applications, it's a pretty bad metric, because it's defined with reference to the relative variation in a particular sample, and completely ignores the absolute deviation of predictions from the true scores (i.e., a model can have an $R^2$ of 1 while generating a predicted score distribution entirely outside the range of the true scores). For our purposes though, $R^2$ is a sensible metric, because for most individual differences variables, absolute scores don't really mean anything anyway—we're almost always interested in how variation across individuals relates to variation in some other measure or construct.

#### How well did we do?
Let's see how well the linear regression model we fitted earlier (using the 5 Big Five domains as predictors) explains the variance in age. We'll make use of scikit-learn's `metrics` module, which contains a large number of predefined performance metrics. As is true of `Estimator` objects, all metrics in scikit-learn follow the same usage pattern: we pass in the true scores and the model's predicted scores, respectively.

In [ ]:
# the metrics module contains predefined scoring functions
# for commonly used metrics like r^2, MSE, etc. but you
# can use any function you like as long as it has the
# signature used in the r2_score call below
from sklearn.metrics import r2_score

# scoring functions are called by passing an array of
# true scores and and an array of predicted scores as
# inputs
r2_score(y, y_pred)

We can explain about 11% of the variance in age using people's Big Five scores. (You can try experimenting with other metrics by replacing `r2_score` with `mean_squared_error`, `mean_absolute_error`, or one of the other predefined metrics. You could even write your own metric function and use it above, as long as it takes the true scores and predicted scores as the sole arguments.)

For convenience, scikit-learn estimators have a `.score()` method you can use as an alternative to the above. Instead of generating predicted scores and then explicitly feeding them to a metric function like `r2_score`, you can call `.score()` directly on the estimator after the `fit()` step, and the prediction will be done implicitly:

In [ ]:
# Initialize the estimator and fit the data, just like before
est = LinearRegression()
est.fit(X, y)
# Now instead of generating predictions explicitly,
# we just call .score(). Note that we lose the ability to
# specify the metric: LinearRegression.score() always uses
# the R^2 metric.
est.score(X, y)

### Classification
Now let's look at classification. In this case, the target labels we're trying to predict are discrete (or categorical). For example, building a model that takes a structural brain image as input and outputs a prediction about whether the brain belongs to a dog or a cat is a classification problem, because the output is discrete: each brain belongs to one of the two classes (or categories), and no brain ever takes on an an intermediate value (though our classifiers can certainly make a graded or probabilistic prediction about which class a brain belongs to).

In practice, we can often turn regression problems into classification problems by discretizing the data in some way. To make the point really clear, let's stick with our personality dataset. Notice that instead of trying to predict continuous age from personality (a regression problem), we *could* in principle bin people into groups based on their age, and then try to develop a model that can correctly *classify* people as "young" or "old" based on their self-reported personality scores. Almost any regression problem can be turned into a classification problem in this way (the converse is not generally true).

I hasten to emphasize that discretizing continuous data for classification purposes is generally *not* a good thing to do (even though it's quite common in many social and biomedical sciences). But we'll allow ourselves the liberty in this particular case, because one thing the act of discretization does make clear is the close relationship between the two kinds of problems.

#### Age prediction as a classification problem
Let's set up our age prediction problems as a classification problem by assigning all participants to either YOUNG (age < 18) or OLD (age >= 40) groups based on their age. I'm well aware that most people are not going to be very happy with a discretization of age that calls anyone over 40 an "old" person. If this bothers you (and it probably should), congratulations: you now appreciate one of the problems (there are several!) with discretizing variables that are naturally continuous.

Let's plot the joint distribution of two personality traits that have a non-trivial relationship with age: Conscientiousness and Extraversion. We'll color the individual points based on the assigned age group.

In [ ]:
# To increase visual clarity, let's subsample
sample = data.sample(20000, axis=0)

# Define age groups based on cut-offs of < 20 and > 40.
# Note: dichotomization of continuous variables is generally a bad idea in the real world!
sample['age_group'] = np.nan
sample.loc[sample['AGE'] < 18, 'age_group'] = 0
sample.loc[sample['AGE'] >= 40, 'age_group'] = 1
sample = sample.dropna(subset=['age_group'])

scatter = plt.scatter(sample['Conscientiousness'], sample['Extraversion'], c=sample['age_group'], s=3, alpha=0.5)
plt.xlabel('Conscientiousness score', fontsize=16)
plt.ylabel('Extraversion score', fontsize=16)
plt.xlim(1, 5)
plt.ylim(1, 5)
plt.grid(False)
plt.legend(*scatter.legend_elements(), loc="lower left", title="Classes");

The classification problem is to identify decision boundaries in our data that maximally separates the two classes—i.e., that puts all the YOUNG participants in one class and all the OLD participants in the other.

#### K-nearest neighbors
There are many classification algorithms we could use for this problem. But let's experiment with K-nearest neighbors (KNN) classification. KNN is a popular approach because it's algorithmically simple and often performs quite well. The core idea is that, for every observation in our test set (i.e., the dataset we want to make predictions for), we identify the $k$ nearest neighbors in the training set (i.e., the set of training observations that are most similar to the target observation in terms of their features), and then assign the class that's most common in that neighborhood. By varying $k$—the size of the neighborhood—we can control the sensitivity of the classifier to local conditions. A small value will yield a highly variable classification that can capture fine-grained variation; a large value will yield smoothly varying boundaries that primarily capture coarse information.

Here's a schematic illustration:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/1920px-KnnClassification.svg.png" alt="KNN" width="600"/>

Let's apply a KNN classifier to our age prediction problem using scikit-learn. First, we pull out the `X` and `y` data from our larger `DataFrame`.

In [ ]:
# Set up our X and y variables
X = sample[['Conscientiousness', 'Extraversion']].values
y = sample['age_group'].values

Now we initialize a `KNeighborsClassifier` and fit it to the data. The `fit`/`predict` procedure looks just like it did for regression problems. Note the key parameter `n_neighbors` (i.e., $k$), which we can experiment with.

In [ ]:
# neighborhood size: experiment with this!
N_NEIGHBORS = 5

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=N_NEIGHBORS, weights='uniform')
knn.fit(X, y)

Let's plot the decision boundaries and see what the classifier is doing...

In [ ]:
# Plot adapted from https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html

from matplotlib.colors import ListedColormap

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
h = .02

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, s=2, alpha=0.6)
plt.xlabel("Conscientiousness score")
plt.ylabel("Extraversion score")
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("KNN classification (k = %i)" % N_NEIGHBORS);

#### How well did we do?
It seems pretty clear from the above plot that performance is reasonably good—i.e., most of the observations are being assigned to the right class. But let's quantify that. As a first pass, we can just look at mean accuracy.

In [ ]:
# Score performance using mean accuracy (i.e., proportion correct)
knn.score(X, y)

This seems pretty decent, but overall accuracy can mask some important nuances. We can get some more information using the `classification_report` utility.

In [ ]:
from sklearn.metrics import classification_report

y_pred = knn.predict(X)
print(classification_report(y, y_pred))

The [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) is also often helpful:

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = knn.predict(X)

cm = confusion_matrix(y, y_pred)

pd.DataFrame(cm, columns=["YOUNG (pred.)", "OLD (pred.)"], index=["YOUNG (true)", "OLD (true)"])

## Unsupervised learning
Learning is unsupervised when there's no ground truth or right answer, and the goal is just to learn some useful structure from the data. The learning algorithm receives no direct guidance about how well it's performing.

For example, suppose someone hands us the following data, with no further explanation:

In [ ]:
# scikit-learn has some handy utilities for generating structure data
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=1, cluster_std=1)
plt.scatter(*X.T);

It would be natural to think that these data are generated from three distinct *clusters*. But since the data are unlabeled, we don't know for a fact that this assignment is correct; we're inferring the grouping in an unsupervised way, based on whatever principle are built into our estimation method (e.g., our visual system's tendency to group objects together based on proximity). The lack of access to the ground truth—and often, it's not even clear that there *is* any ground truth—underscores the difficulty of the unsupervised learning challenge.

#### k-means clustering
It should not surprise you at this point to learn that doing unsupervised learning in scikit-learn is pretty easy (which is not to say that doing it *well* is easy, of course). The code looks almost exactly the same is it does in the unsupervised setting. The main difference is that unsupervised learning estimators take only `X` data—there is, by definition, no set of labels `y`.

Let's cluster the observations in the above plot using the [k-means clustering](https://en.wikipedia.org/wiki/K-means_clustering) algorithm:

In [ ]:
from sklearn.cluster import KMeans

kmc = KMeans(3)
kmc.fit(X)

colors = kmc.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=colors);

### Job well done?
At this point we've developed fully operational machine learning workflows for three separate kinds of machine learning problems: regression, classification, and clustering. Relatively little work was involved in each case. In the case of the regression problem, for example, we initialized a linear regression model, fit it to some data, used it to generate predictions, and scored those predictions— all in 3 or 4 lines of code! This seems pretty great. Maybe we should just stop here, pat ourselves on the back for a job well done, and head home for the day.

For reasons that will shortly become clear, though, calling it quits here would be a really bad idea.

Let's dig a little deeper. We'll start by asking an important question that echoes back to the definition of machine learning as the study of systems that can improve their output by learning from experience. Specifically: how does our model's performance evolve as we give it more data?

Think back to our (continuous) age prediction problem. Intuitively, we might expect that our $R^2$ will go up as we increase the size of our dataset (because having more data to learn from seems like it should be a good thing). But we should probably verify that.

Instead of fitting our `LinearRegression` estimator to just one dataset, let's systematically vary our sample size over a large range, and fit a linear regression to each one (actually, to stabilize our performance estimates, we'll average over multiple permutations at each sample size).

The code below is much more involved than it needs to be; as we'll see in the next section, scikit-learn includes a number of utilities that can achieve the same goal much more compactly and efficiently. But I think it can be helpful to explicitly lay out all of the steps we're going through before we replace them with a single line of black magic.

In [ ]:
# initialize the OLS estimator
est = LinearRegression()

# we'll plot a separate panel for each feature set
feature_sets = ['domains', 'facets', 'items']

# evaluate performance at each of these sample sizes
sample_sizes = [100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000]

# number of permutations to average over at each sample size
n_reps = 10

# store results for all permutations, sample sizes, and feature sets
results = np.zeros((n_reps, len(sample_sizes), len(feature_sets)))

# loop over permutations
for i in range(n_reps):
    # loop over sample sizes
    for j, n in enumerate(sample_sizes):
        # get the appropriate features and labels
        *Xs, age = get_features(data, *feature_sets, 'AGE', n=n)
        # loop over feature sets
        for k, X in enumerate(Xs):
            # fit the model
            est.fit(X, age)
            # generate predictions
            pred_y = est.predict(X)
            # save R^2 in our results array
            results[i, j, k] = r2_score(age, pred_y)

# Compute means and stdevs for error bars 
r2_mean = results.mean(0)
r2_std = results.std(0)

Now we can plot the resulting $R^2$ values for each feature set as a function of sample size:

In [ ]:
# used to display axis tick labels on a linear scale
from matplotlib.ticker import ScalarFormatter

# Set up plots=
fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)
plt.ylim(0, 1)

# Plot results
for i, label in enumerate(feature_sets):
    mean, sd = r2_mean[:, i], r2_std[:, i]
    ax = axes[i]
    line = ax.plot(sample_sizes, mean, 'o-')
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel("Sample size (n)", fontsize=14)
    ax.fill_between(sample_sizes, mean-sd, mean+sd, alpha=0.2)
    ax.set_title(label, fontsize=18)

# Add y-axis labels on both sides
axes[0].set_ylabel("$R^2$", fontsize=16);
axes[-1].set_ylabel('$R^2$', fontsize=16)
axes[-1].yaxis.set_label_position("right")

Here, each panel displays the model's ability to predict (on the y-axis) as a function of sample size (x-axis). Each panel displays results for a different set of predictive features: at left, we have the 5 domains; in the middle, the 30 facets; and at right, the 300 items. This gives us some insight into how model performance varies as we add more features—something we'll discuss in much more detail in the next couple of notebooks.

For now, let's focus on the dominant pattern we see within each of the panels. Strangely, it looks like model performance *decreases* with increasing sample size! You may find this odd, seeing as everyone's always talking about the wonders of Big Data, and intuitively, you might think that having more information available would lead the model to make *better* predictions.

Actually, the model probably *is* making better predictions when it has more data to learn form. It just doesn't look that way. The problem is not what's happening on the right side of the curve, but what's happening on the left side (i.e., when sample size is small). On the left, the model is *overfitting* the data. Because there are very few data points, and our model is extremely flexible (it has 300 degrees of freedom with which to predict only a few hundred points!), there's nothing to stop the model from learning noise rather than signal. As the sample size grows, the data provide a natural buffer against this kind of thing. We'll explore this idea in much more detail in the next couple of sections.